In [1]:
from __future__  import print_function  # Python 2/3 compatibility

import pkg_resources
import json
import pandas as pd
import boto3
pkg_resources.get_distribution("boto3").version

'1.19.3'

In [2]:
s3 = boto3.client('s3')

response = s3.get_object(
    Bucket='bearly-graph',
    Key='src/mpd.slice.0-999.json',
)
# Extracts body of the 
body = response['Body']
str_body = body.read()
s3_obj_content = json.loads(str_body)
playlists = s3_obj_content['playlists']

In [3]:
artist_list = []

for playlist in playlists:
    tracks = playlist['tracks']
    for track in tracks:
        artist_dict = {}
        artist_dict['~id'] = track['album_uri']
        artist_dict['album_name:String'] =  track['album_name']
        artist_dict['label'] =  'album'
        artist_list.append(artist_dict)

df = pd.DataFrame.from_dict(artist_list)

    

In [4]:
df.drop_duplicates()

,~id,album_name:String,label
0,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook,album
1,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,In The Zone,album
2,spotify:album:25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit),album
3,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justified,album
4,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Hot Shot,album
...,...,...,...
67496,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,Roots,album
67499,spotify:album:5jrsRHRAmetu5e7RRBoxj7,"Hollywood, a Beautiful Coincidence",album
67500,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,Roadhouse 01,album
67501,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,Good Feeling,album


In [5]:
b_string = df.to_csv().encode('utf-8')
s3.put_object(Body=b_string, Bucket='bearly-graph', Key='vertex/version=2/album_vertex_0.csv')

{'ResponseMetadata': {'RequestId': 'F81F0EJAD9HD6V81',
  'HostId': 'UAhq5PP+0A7fuX6Kyl/MoO3q08iTuWy2c9sf58CtjXheGKF4vYAG5EqDdNC4u32uvJ7MjRiWUT0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'UAhq5PP+0A7fuX6Kyl/MoO3q08iTuWy2c9sf58CtjXheGKF4vYAG5EqDdNC4u32uvJ7MjRiWUT0=',
   'x-amz-request-id': 'F81F0EJAD9HD6V81',
   'date': 'Tue, 02 Nov 2021 04:03:18 GMT',
   'etag': '"621609b475d43bdbcfb6db49d79a2d0c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"621609b475d43bdbcfb6db49d79a2d0c"'}